In [ ]:
import pandas as pd
import os
df = pd.read_csv('/Users/carterhogan/CaseStudies/world_value_survey/analysis/data/wvs/WVS_Cross_National.csv')

/var/folders/k9/j4977wt918z9ns7wk6kqfmqm0000gn/T/ipykernel_3862/621221244.py:3: DtypeWarning: Columns (346,348,350,580,581,582,583) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/carterhogan/CaseStudies/world_value_survey/analysis/data/wvs/WVS_Cross_National.csv')


In [ ]:
# Filter for Countries in "Western Europe" as defined by the European Union (https://eur-lex.europa.eu/browse/eurovoc.html?params=72,7206,913#arrow_913)
w_europe = [20, 40, 250, 276, 528, 756, 826 ]
# Belgium, Ireland, Liechtenstein, Luxembourg, and Monaco do not have data in wave 7
df = df[df['B_COUNTRY'].isin(w_europe)]

In [32]:
import openpyxl
# Read in the WPF Index 
os.chdir('/Users/carterhogan/CaseStudies/world_value_survey/analysis/data/additional/')
wpf = pd.read_excel('RWB-PFI.xlsx')

In [39]:
countries = ['AND','AUT','CHE','FRA','GBR','DEU','NLD']
wpf = wpf[wpf['Economy ISO3'].isin(countries)]
wpf = wpf[wpf['Indicator ID'] != 'RWB.PFI.RANK']
wpf = pd.concat([wpf.iloc[:, :2], wpf.iloc[:, -7:]], axis=1)


In [ ]:
# Select the first two columns and the last seven columns
main_columns = wpf.iloc[:, :2]  # First two columns
year_columns = wpf.iloc[:, -7:]  # Last seven columns

# pivot the last seven columns into rows so that we can merge into broader df
pivoted_df = pd.melt(
    wpf, 
    id_vars=main_columns.columns,  # Columns to keep (e.g., the first two columns)
    value_vars=year_columns.columns,  # Columns to pivot (e.g., the last seven columns)
    var_name='Year',  # Name of the new column that will hold column names
    value_name='WPFI'  # Name of the new column that will hold the values
)


In [ ]:
# Perform the left join on 'ISO' and 'Year'
merged_df = pd.merge(
    df, 
    pivoted_df, 
    how='left', 
    left_on=['B_COUNTRY_ALPHA', 'A_YEAR'], 
    right_on=['Economy ISO3', 'Year']
)

# Now we have the world press freedom index for each country for the year in which it was interviewed

In [55]:
#Write the wave 7 data to a finalized df
merged_df.to_csv('/Users/carterhogan/CaseStudies/world_value_survey/analysis/data/wvs/wave7.csv')



In [56]:
# Now we need to get the data for the time series

ts = pd.read_csv('/Users/carterhogan/CaseStudies/world_value_survey/analysis/data/wvs/WVS_Time_Series.csv')

In [58]:
ts = ts[ts['COUNTRY_ALPHA']== 'DEU']


In [ ]:
# Due to the limitation in years, it is hard to 
# decide cutting out 1997 for the purpose of using WPFI (thoughts?)
ts['S020'].unique()

array([1997, 2006, 2013, 2018])